In [ ]:
# after taking boitier back from Coumarane
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))



import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']


from glob import glob
from pathlib import Path
fnbs = []
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
for fnf in glob(d + '2023-SE1-0*[0-9].log'):
    name = Path(fnf).name
    fnb = name.replace('.log','')
    fnbs += [fnb]
    #print( f'"{fnb}",' )
fnbs = list(sorted(fnbs))
display(fnbs)
    
    
#fnbs = ['2023-SE1-010_context_change_20230413_172011']
fnbs = ['2023-SE1-009_context_change_20230413_101827']
print(fnbs)  

In [ ]:
#params['width']

In [ ]:
    
dfs = []
for fnb in fnbs:
    fn = fnb + '.log'
    fnp = fnb + '.param'

    with open(pjoin(d,fn), 'r') as f:
         l = f.readline()
         truelen = len( l.split(',') )

    r = ('trial_index, current_phase_trigger, tgti_to_show,'
       ' vis_feedback_type, trial_type, special_block_type, block_ind, '
        ' feedbackX, feedbackY, unpert_feedbackX, unpert_feedbackY,'
         ' error_distance, target_coordX, target_coordY, '
         'feedbackX_when_crossing, feedbackY_when_crossing, '
         'jax1, jax2, reward, time')
    r = r.replace(' ','')
    colnames = r.split(',')
    print('len(colnames) = ',len(colnames),'truelen = ',truelen,colnames)

    nbad = 3  # recompense strip
    #colnames = ['trial_index', 'current_phase_']
    df = pd.read_csv(pjoin(d,fn), skipfooter = nbad, on_bad_lines='warn', header=0,
                    names=colnames, encoding='latin1')
    #encoding='latin-1'
    subj = fnb.split('_')[0]    
    df['subject'] = subj
    dfs += [df]
    
df_unproc = pd.concat( dfs ).reset_index()

print('max trial index = ', df_unproc['trial_index'].max() )

In [ ]:
dfs_full = dfs

In [ ]:
NT = 10,40
dfs_r = []
for df in dfs:
    dfs_r += [df.query('trial_index >= @NT[0] and trial_index <@NT[1]')]
dfs = dfs_r

In [ ]:
dfs = dfs_full

In [ ]:
df_unproc = pd.concat( dfs ).reset_index()
df_unproc = df_unproc.drop(labels=['index'],axis=1)
#df_unproc

from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

dfcs = []; dfccs = []; dfcpcs = []; dfctcs = [];    
for df in dfs:
    print('---------------  Starting processing new df')                    
    dfc, dfcc, dfcpc, dfctc = addBasicInfo(df, phase2trigger, params,
                                         home_position, target_coords)


    dfcc['nonhit'] = dfcc['error_distance'] > hitr
    dfcs += [dfc]; dfccs += [dfcc]; dfcpcs += [dfcpc]; dfctcs += [dfctc];   

dfcc_all  = dfcc
dfc_all   = dfc
dfcpc_all = dfcpc
dfctc_all = dfctc

subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[0]
print(subj)
dfccos  = dfcc_all.query('subject == @subj')
dfcos   = dfc_all.query('subject == @subj')
dfcpcos = dfcpc_all.query('subject == @subj')
#dfcc = dfccos
##########################
a = dfcc_all['error_area_signed'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_area_signed_nonhit'] = a > thr

a = dfcc_all['error_area_signed_scaled_ed'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_area_signed_scaled_ed_nonhit'] = a > thr

a = dfcc_all['error_lh_ang'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_lh_ang_nonhit'] = a > thr

a = dfcc_all['error_area_signed_nn_scaled_ed'].abs()
thr = a.quantile(5e-2); print(thr)
dfcc_all['error_area_signed_nn_scaled_ed_nonhit'] = a > thr

a = dfcc_all['error_area2_signed_nn_scaled_ed'].abs()
thr = a.quantile(5e-2); print(thr)
dfcc_all['error_area2_signed_nn_scaled_ed_nonhit'] = a > thr
##############################
assert 'error' not in dfcc_all.columns
df_all = dfcc_all.drop(labels=['index', 'level_0'],axis=1)  #.copy()
#df_all['non_hit_not_adj'] = df_all['nonhit']
df_all['target_inds']     = df_all['tgti_to_show']
df_all['trials']          = df_all['trial_index']

# for err sens calc
#df_all['error']          = df_all['error_area_signed']
##############################
subjects_ = list( df_all['subject'].unique() )
print(subjects_)



# Look at trial seq

In [ ]:
trigger2phase

In [ ]:
df_end = df_unproc.groupby('trial_index').max('time')

In [ ]:
df_end

In [ ]:
dfcc[['trial_index','tgti_to_show', 'error_area_signed_nn', 'error_area2_signed_nn','error_distance']]

# 2d plots

In [ ]:
s = subjects[0]

In [ ]:
grp_perti = dfcos.groupby(['trial_index'])

pause_inds = dfcpc_all.query(f'subject == "{s}"').groupby('trial_index').max('time').index
#['trial_index']
pause_inds = np.array(pause_inds)

#rngs = f'{rng.start,rng.stop}'
#rngs = f'S={S}; N={N}'

In [ ]:
rng = np.arange(0,36)
rngs = f'{rng[0]}-{rng[-1]}'; print(rngs)

N = len(rng); print(N)

In [ ]:
dfesqs = (f'subject == "{s}"')    
dfcc.query(dfesqs)

In [ ]:
from exper_protocol.utils import screen2homec
target_coords_homec = screen2homec( *tuple(zip(*target_coords)), home_position  )

In [ ]:
colscols = [['trialwb','perturbation','trial_type'],
            ['prev_trial_type'],
            ['error_endpoint_ang', 'error_lh_ang'],            
            ['error_distance','time_mvt', 'traj_length'],
            ['error_area_signed_nn'],
            ['error_area_ofb_signed_nn', 'nonhit']]

In [ ]:
#N  = len(rng) //2; N

In [ ]:
from behav_proc import plotTraj, plotTraj2
from config2 import path_fig
#rng = range(600,603)
#rng = range(596,625)
#rng = range(8,10)
#rng = range(50,60)
#rng = range(55,57)

show_dist_guides = False
ww = 7; hh =10
nc = int(np.ceil(np.sqrt(len(rng)) ) ); nr = int( np.ceil( len(rng) / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi2d = {}
for axi,ti in enumerate(rng):
    ax = axsf[axi]    
    if ti not in grp_perti.groups:
        if ti in pause_inds:            
            print(f'Skipping {ti} because pause')
            ax.set_title('PAUSE')
        else:
            print(f'Skipping {ti} because ?')
        continue
        
    df0 = dfcc.query(f'subject == "{s}" and trial_index == {ti}')
    row = df0.to_dict('records')[0]
    
    ev = row['error_endpoint_ang']
    tiwb = row['trialwb']
    
    #dfes = df_all_multi_tsz.query(dfesqs + f' and trial_index == {ti}')
    dfes = dfcc.query(dfesqs + f' and trial_index == {ti}')
    assert len(dfes) == 1
    row2 = dfes.to_dict('records')[0]
    row.update(row2)
    
    
    idx = grp_perti.groups[ti]
    dfcurtr = dfcos.loc[idx[1:]]  # zeroth is equal to zero
    import sys, traceback
    try:
#         plotTraj2(ax, dfcurtr, home_position, target_coords_homec, params,
#                 force_entire_traj=0, calc_area = 1, addinfo=row,
#                 titlecols = colscols, xlim=(-160,160))
        
        r = row2multierr(row, dfc, grp_perti, home_position,             
            target_coords, params, ax=ax,
            revert_pert = False, xlim=(-160,160),
            titlecols = colscols, addinfo=row)
        z = zip( ['err', 'error_lh_ang', 'error_unpert_lh_ang', 'start_dist',
                      'traja', 'traja2', 'trajoa', 'td',
                'length',  'enddist'], r)
        dd = dict(z)
    
    except Exception as e:
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)

        psf = exc_info[2].tb_next
        stackframe = exc_info[2].tb_next.tb_frame

        display(psf.tb_frame)
        #display(psf.tb_next.tb_frame)

        local_vars_in_fun = stackframe.f_locals
        plt.close()
        break
    
    from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.set_yticks(np.arange(-50,400,50))
    #ax.xaxis.set_minor_locator(AutoMinorLocator(25))
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    #ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    axi2d[axi] = {'df0':df0, 'row':row, 'dfcurtr':dfcurtr}
    
    print(f'error = {ev}, tiwb = {tiwb}')
    
plt.tight_layout()
plt.savefig(pjoin(path_fig, f'traj_2D_{row["subject"]}_{rngs}.pdf') )
plt.close()

In [ ]:
# display(psf.tb_frame)
# lf = psf.tb_frame.f_locals
# if psf.tb_next is not None:
#     psf2 = psf.tb_next
#     display(psf2.tb_frame)
#     lf0 = lf
#     lf = psf2.tb_frame.f_locals

# #display(exc_info[2].tb_frame)

In [ ]:
df_unproc['jax2'].describe()

In [ ]:
df_unproc['jax1'].describe()

In [ ]:
ww = 5; hh =3
nc = int(np.ceil(np.sqrt(len(rng)) ) ); nr = int( np.ceil( len(rng) / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi2d = {}
for axi,ti in enumerate(rng):
    ax = axsf[axi]    
    if ti not in grp_perti.groups:
        if ti in pause_inds:            
            print(f'Skipping {ti} because pause')
            ax.set_title('PAUSE')
        else:
            print(f'Skipping {ti} because ?')
        continue
        
    df0 = dfcc.query(f'subject == "{s}" and trial_index == {ti}')
    row = df0.to_dict('records')[0]
    
    ev = row['error_endpoint_ang']
    tiwb = row['trialwb']
    
    #dfes = df_all_multi_tsz.query(dfesqs + f' and trial_index == {ti}')
    dfes = dfcc.query(dfesqs + f' and trial_index == {ti}')
    assert len(dfes) == 1
    row2 = dfes.to_dict('records')[0]
    row.update(row2)
    
    
    idx = grp_perti.groups[ti]
    dfcurtr = dfcos.loc[idx[1:]]  # zeroth is equal to zero
    
    jax1 = dfcurtr['jax1'].to_numpy()
    jax2 = dfcurtr['jax2'].to_numpy()
    #ax.scatter(jax1,jax2)
    times = dfcurtr['time_since_trial_start'].to_numpy()
    ax.plot(times,jax1, label='jax1')
    ax.plot(times,jax2, label='jax2')
    ax.legend(loc='lower left')
    ax.set_title(f'trial_index={ti}')
    print(np.max(jax1), np.min(jax1), np.std(jax1))
    
plt.tight_layout()
plt.savefig(pjoin(path_fig, f'traj_jax_{row["subject"]}_{rngs}.pdf'))

In [ ]:
ax= plt.gca()
jax1 = dfcurtr['jax1'].to_numpy()
jax2 = dfcurtr['jax2'].to_numpy()
#ax.scatter(jax1,jax2)
times = dfcurtr['time_since_trial_start'].to_numpy()
ax.plot(times,jax1)
ax.plot(times,jax2)